In [42]:
import pandas as pd
from main import *


train_data = pd.read_csv('./data/train_data.csv', index_col=0)
label_denotation = {
    1: 'positive',
    0: 'negative'
}
train_data.sample(10)

,text,label
id,,
741,爱人之间来不得半点儿虚伪，一旦开始，就覆水难收。,positive
145,觉得很平淡，在背景为导演是希区柯克的情况下。,negative
221,一句话：难看到家了！！唯唯诺诺的讲，实在胆小鬼，那么何必要拍呢！！,negative
215,王力宏很帅，这不太像是一部电影，像童话故事或者别的什么,negative
177,嘛叫烂片，这就是。。。浪费时间，浪费钱，糟蹋演员，也折磨观众。。。,negative
655,不错的喜剧片，彭导真的厉害啊，这片子各种奇妙的构思。,positive
435,我也不知道看了点什么，印象中付辛博演娘炮演得还不错。,negative
102,和 【我唾弃你的坟墓3】极度相似， 女主眼睛挺有特点的确实像个天使。,negative
680,相信天堂的存在很幸福，会无惧死亡，还会见到以你幻想的样子出现的上帝。,positive


In [43]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_data, test_size=0.17, random_state=12)
print(train.shape, val.shape)

(830, 2) (170, 2)


In [46]:
databunch = TextClasDataBunch.from_df(".", train, val,
                  tokenizer=fastai_tokenizer,
                  vocab=fastai_bert_vocab,
                  include_bos=False,
                  include_eos=False,
                  text_cols="text",
                  label_cols='label',
                  bs=config.bs,
                  collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
             )

In [52]:
databunch.show_batch()

text,target
[CLS] 我 对 该 剧 组 的 化 妆 师 觉 得 很 好 奇 。 一 定 要 那 么 厚 的 粉 ？ 那 么 不 自 然 的 腮 红 ？ 明 明 有 些 场 景 很 漂 亮 。 苦 了 那 小 花 店 ~ [SEP],negative
[CLS] 不 禁 想 到 自 己 非 常 非 常 年 迈 时 会 怎 样 看 待 所 谓 生 命 的 意 义 和 这 一 生 的 旅 程 ， 是 否 还 有 力 量 改 变 ， 或 者 释 然 的 一 笑 。 [SEP],positive
[CLS] 女 主 真 是 尤 物 啊 ， 结 局 被 黑 色 幽 默 了 一 把 ， 和 整 部 片 子 的 冷 幽 默 基 调 一 致 ， 安 娜 卡 列 宁 娜 出 场 也 是 神 来 之 笔 哈 哈 [SEP],positive
[CLS] 有 乱 伦 来 降 低 师 生 恋 的 不 堪 ， 看 完 后 ， 果 真 是 三 观 尽 失 。 类 似 的 情 况 ， 男 主 像 英 雄 ， 儿 媳 像 婊 子 。 什 么 狗 屁 片 子 [SEP],negative
[CLS] 79 年 的 电 影 ， 纯 洁 的 心 灵 ， 纯 粹 的 精 神 。 看 腻 了 好 莱 坞 和 所 谓 的 国 产 大 片 ， 再 看 这 些 老 电 影 顿 时 觉 得 神 清 气 爽 。 [SEP],positive


In [55]:
from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification
bert_model = BertForSequenceClassification.from_pretrained(config.bert_model_name, num_labels=2)

In [58]:
loss_func = nn.CrossEntropyLoss()

In [65]:
learner = Learner(databunch, bert_model, loss_func=loss_func, metrics=accuracy)

In [68]:
learner.load("./stage1")

Learner(data=TextClasDataBunch;

Train: LabelList (830 items)
x: TextList
[CLS] 故 事 真 的 没 有 一 点 儿 新 的 点 。 编 剧 可 是 邱 刚 健 啊 [UNK] [UNK] 剪 辑 也 挺 二 的 ， 废 戏 太 多 。 [SEP],[CLS] . . . . . . . . . . . . . . 为 毛 石 像 鬼 竟 然 是 天 使 啊 [SEP],[CLS] 史 诗 什 么 的 被 好 莱 坞 盯 上 一 般 都 是 稳 遭 毒 手 ， 一 颓 到 底 的 。 [SEP],[CLS] 为 什 么 那 时 候 的 车 戏 看 起 来 那 么 逼 真 ， 现 在 电 视 剧 里 车 上 的 玻 璃 外 面 明 显 是 贴 的 纸 [SEP],[CLS] 色 彩 饱 满 ， 非 洲 口 音 萌 萌 哒 ， 结 尾 那 个 无 厘 头 的 终 点 根 本 不 是 终 点 好 吗 ！ [SEP]
y: CategoryList
negative,negative,negative,positive,positive
Path: .;

Valid: LabelList (170 items)
x: TextList
[CLS] 经 典 啊 ！ 我 初 中 看 电 视 播 了 片 段 就 记 忆 深 刻 了 [SEP],[CLS] 这 个 故 事 告 诉 我 们 ， 贼 可 以 爱 上 贼 ， 但 贵 妇 却 不 行 ～ [SEP],[CLS] 如 果 不 是 那 个 破 坏 别 人 婚 礼 现 场 的 糟 糕 的 结 尾 ， 这 其 实 是 一 部 还 不 错 的 片 子 。 [SEP],[CLS] 一 场 闹 剧 ， 又 不 好 笑 又 无 营 养 ， 菊 花 变 成 向 日 葵 [SEP],[CLS] 蓝 色 是 忧 郁 的 颜 色 ， 片 子 演 技 有 些 过 了 ， 没 看 完 ， 看 不 下 去 [SEP]
y: CategoryList
positive,positive,positive,negative,negative
Path: .;

Test: None, model=BertForSequenceClassification(
  (bert): 

In [75]:
learner.predict("设施老化，紧靠马路噪音太大，晚上楼上卫生间的水流声和空调噪音非常大，无法入眠")[2]

tensor([0.9715, 0.0285])